In [25]:
from urllib.parse import urlparse
import requests

import scrapy

In [185]:
def get_event_json(url):
    url_parts = urlparse(url)
    host = url_parts.netloc
    print(host)
    print('---')
    page = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:53.0) Gecko/20100101 Firefox/53.0'})
    dom = scrapy.Selector(text=page.content)
    if host == 'www.facebook.com':
        title = dom.css('#seo_h1_tag ::text').extract_first()
        datetime = dom.css('#event_summary').extract()
        json = {
                'title': title,
                'datatime': 'unheard',
                'location': 'na kudykinoy gore',
                'source': 'facebook'
               }
    if host == 'events.yandex.ru':
        json = {
            'title': dom.css('h2.title ::text').extract_first(),
            'datetime': dom.css('.event-header__when ::text').extract_first(),
            'location': dom.css('.event-header__place ::text').extract_first() + ' Яндекс',
            'source': url,
            'descripton': dom.css('.b-static-text ::text').extract()
        }
    if host == 'www.meetup.com':
        json = {
            'title': dom.css('h1[itemprop="name"] ::text').extract_first().strip(),
            'datetime': dom.css('.past-event-info li ::text').extract_first().strip(),
            'location': dom.css('.past-event-info li[data-name] ::attr(data-name)').extract_first().strip() + ', ' + dom.css('.past-event-info li[data-name] ::attr(data-address)').extract_first().strip(),
            'source': url,
            'decription': dom.css('#past-event-description-wrap ::text').extract()
        }
    if host.endswith('timepad.ru'):
        json = {
            'title': dom.css('.ep-3-hero__subtitle ::text').extract_first().strip(),
            'datetime': dom.css('.ep3-pagesummary__time-begin span ::text').extract_first().replace('\xa0', ' ').strip(),
            'location': dom.css('.ep3-pagesummary__place-city ::text').extract_first().strip() + ', ' + dom.css('.ep3-pagesummary__place-adress span ::text').extract_first().strip(),
            'source': url,
            'description': dom.css('.ep3-content .clearfix p ::text').extract()
        }
    return json